### Analyzing Some Ocean Model Output

Below is a some code that uses the [Pangeo stack](https://pangeo.io/) to read in
output from the ocean component of the CESM model. This run was done on a
nominally 3-degree resolution grid with 11600 total cells. The output file
contains monthly averages for two variables: sea surface temperature (`SST`) and
air-sea CO2 flux (`FG_CO2`). The area of each grid cell is also included
(`TAREA`).

We use [xarray](http://xarray.pydata.org/en/stable/) to read in the data, but
you don't need to know anything about that package. There are a few
[numpy](https://numpy.org/) functions that will be useful, and we provide an
example of them below.


In [ ]:
import numpy as np
import xarray as xr

In [ ]:
ds = xr.open_dataset("SST_and_CO2FLUX.nc")
ds

#### What does this data look like?

`xarray` can plot data using `matplotlib`. For example, below is a plot of the
January CO2 flux.


In [ ]:
ds.isel(time=0)["FG_CO2"].plot()

Note that the white coloring for land is a `nan` value rather than a 0. The
following cell uses some `numpy` functions to show that there are ~3700 land
cells: `isnan` returns `True` where the array is `nan` rather than real-valued
and `False` elsewhere; `where` creates an array that is 1s where the condition
is true and 0 where it is false.


In [ ]:
np.sum(np.where(np.isnan(ds.isel(time=0)["FG_CO2"].values), 1, 0))

The units of the `FG_CO2` variable, shown in the caption of the plot above, is `mmol/m^3 cm/s`:

In [ ]:
ds["FG_CO2"].attrs["units"]

That looks a little odd, but it's a concentration (`mmol/m^3`) times a velocity (`cm/s`).
Note that this is moles per area (`cm / m^3` is 1 / area) per time.

A common question to ask is "how much carbon is entering (exiting) the ocean in a given year?"
To answer that, we need to integrate `FG_CO2` over the globe, resulting in units of moles per time.

#### Unit Conversion

There will be a few different conversion factors at play.
First, we want to resolve `cm / m^3` into units that match the reciprocal of the units of the area of our cells:

In [ ]:
ds["TAREA"].attrs["units"]

We also want to convert our unit of time from `s` to `years` and `mmol` to `Pg C`.

Below we have started to write a function to convert an array with units `mmol/m^3 cm/s` to `Pg / cm^2 / yr`.
All that is missing is the conversion factor itself, please determine the proper value and update the function.

Notes:

1. A mole of carbon dioxide contains 12 grams of carbon
1. A petagram is $10^{15}$ grams
1. In our model, each year is exactly 365 days.

In [ ]:
def convert_concentration_units(da):
    """
    Given a DataArray with units mmol/m^3 cm/s, convert the units to Pg / cm^2 / yr.

    This function returns a DataArray with the updated units.
    """
    # (1) Check the units of the argument in
    #     i. If it is already "Pg / cm^2 / yr" return
    units = da.attrs["units"]
    if  units == "Pg / cm^2 / yr":
        return da
    #     ii. If the units are not "mmol/m^3 cm/s", abort
    if units != "mmol/m^3 cm/s":
        raise ValueError(f"Can not convert from '{units}'")

    # (2) Pull numpy array of values out of xarray DataArray, and create a new DataArray to return
    old_values = da.values
    da_out = da.copy()

    # (3) Apply unit conversion
    # TODO: compute the conversion factor
    # conv_factor = 
    new_values = old_values * conv_factor
    da_out.values = new_values
    da_out.attrs["units"] = "Pg / cm^2 / yr"

    # (4) Return new array
    return da_out


In [ ]:
ds["FG_CO2"] = convert_concentration_units(ds["FG_CO2"])

#### Computing a global integral

Below we have started to write a function that approximates the global integral of a field by computing the Riemann sum (i.e. multiplying each cell value by its area and summing). The function loops over the each time dimension, but as you can see it returns `nan` instead of a reasonable value.

We think it might have something to do with the land mask. Can you fix this function?

In [ ]:
def compute_integral(da, area_da):
    """
    Given a DataArray of values (da) and a DataArray of cell areas (area_da),
    returns a DataArray with values sum(da * area_da).

    Returns an xarray DataArray with a single dimension "time".
    """
    # (1) Set up an array for the global integral of each time slice
    #     and pull out numpy array of cell areas
    global_integral = []
    cell_areas = area_da.values

    # (2) For each time slice, compute the global mean
    for n in range(len(ds["time"])):
        # i. pull out numpy array containing correct month of data
        data_to_avg = da.isel(time=n).values

        # ii. integrate
        integral = np.sum(data_to_avg * cell_areas)

        # iii. error checking
        if np.isnan(integral):
            raise ValueError("The integral is nan!")

        # iv. append integral to list
        global_integral.append(integral)

    # (3) convert list to xarray DataArray and get units right
    da_out = xr.DataArray(global_integral, dims="time", coords={"time": ds["time"]})
    if da.attrs["units"] == "Pg / cm^2 / yr" and area_da.attrs["units"] == "centimeter^2":
        da_out.attrs["units"] = "Pg / yr"
    else:
        da_out.attrs["units"] = f"{da.attrs['units']} {area_da.attrs['units']}"

    # return DataArray
    return da_out

In [ ]:
compute_integral(ds["FG_CO2"], ds["TAREA"]).plot()

#### Average SST

If we plot the January SST values, we see that temperatures range from -2 C to a little over 32 C

In [ ]:
print(f"Min SST: {ds['SST'].min().values}")
print(f"Max SST: {ds['SST'].max().values}")
ds["SST"].isel(time=0).plot(vmin=-2, vmax=33)

However, if we plot the integral of SST then we end up with large values and weird units:

In [ ]:
compute_integral(ds["SST"], ds["TAREA"]).plot()

What we actually want to compute is a weighted mean of the SST, or sum(SST * weights)/sum(weights).

This is very similar to the compute_integral function, so I copied my code over.
I still need help fixing the `nan` bug, but something else seems to be going wrong because I know the average SST from this run is around 18 degrees.

In [ ]:
def compute_weighted_mean(da, area_da):
    """
    Given a DataArray of values (da) and a DataArray of cell areas (area_da),
    returns a DataArray containing the weighted mean of da.

    Returns an xarray DataArray with a single dimension "time".
    """
    # (1) Set up an array for the global mean of each time slice
    global_mean = []

    # (2) For each time slice, compute the global mean
    for n in range(len(ds["time"])):
        # i. pull out numpy array containing correct month of data
        #    also compute the weights for the weighted mean
        data_to_avg = da.isel(time=n).values
        weights = area_da.values

        # ii. compute mean
        mean = np.sum(data_to_avg * weights) / np.sum(weights)

        # iii. error checking
        if np.isnan(mean):
            raise ValueError("The mean is nan!")

        # iv. append mean to list
        global_mean.append(mean)

    # (3) convert list to xarray DataArray (note that units are unchanged from original)
    da_out = xr.DataArray(global_mean, dims="time", coords={"time": ds["time"]})
    da_out.attrs["units"] = da.attrs['units']

    # return DataArray
    return da_out

In [ ]:
compute_weighted_mean(ds["SST"], ds["TAREA"]).plot()